In [2]:
import pandas as pd

We have created a 20% overlap for the samples to be labeled to assess the inter-rated agreement. Since annotator vary for each label duplicate pair, we can't use the sklearn's solution as it assumes that raters are the same in each case. Averaging the sklearn's  cohen_kappa_score along the pairs is not a viable solution since if you only have one data point per rater pair, the Cohen's kappa score calculation will result in a warning because it expects more variation in the data (or will return 0 if values are different).

We have calculated the Cohen's kappa for the dataset with the (wrong) assumption that there are only two annotators.
Further we improved the calculation with averaging the pairs, for that we have used this [solution](https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3) instead.

In [235]:
# https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3
def cohen_kappa(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations provided by first annotator
    :type ann1: list
    :param ann2: annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

# Loading and preprocessing the data

In [3]:
df_labels = pd.read_json('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/labels_ready.json')

In [234]:
import json
def extract_class_pairs(json_lst):
    pairs = []
    for json_str in json_lst:
        question_title = json_str['title']
        answer_title = json_str['answer']['title']
        pairs.append([question_title,answer_title])
    return pairs

In [54]:
df_labels['pairs'] = df_labels['Label'].apply(lambda x: extract_class_pairs(x['classifications']))

In [184]:
df = df_labels[['Created By', 'pairs', 'External ID']]

In [185]:
value_counts = df['External ID'].value_counts()
unique_rows = df[df['External ID'].isin(value_counts[value_counts == 1].index)]
df.drop(unique_rows.index, inplace=True)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/3128516445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(unique_rows.index, inplace=True)


In [187]:
for i in range(3):
    col_name = 'pair{}'.format(i+1)
    df[col_name] = df['pairs'].apply(lambda x: x[i] if len(x) > i else None)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/262876240.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = df['pairs'].apply(lambda x: x[i] if len(x) > i else None)
/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/262876240.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = df['pairs'].apply(lambda x: x[i] if len(x) > i else None)
/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_11694/262876240.py:3: SettingWithCopyWarning: 
A valu

In [188]:
df = df[~df['pair1'].isna()]
df.reset_index(inplace=True, drop=True)

In [190]:
text2 = df['pair2'].iloc[9]
text2

['Чи прирівнюються українці до неістот, тварин чи людей, позбавлених людських рис (частково або повністю)?',
 'ні']

In [191]:
text3 = df['pair3'].iloc[9]
text3

['Чи присутня в тексті емоційна оцінка українців?', 'ні, оцінка не присутня']

In [192]:
def replace_with(cell, replace_list):
    if cell is None:
        return replace_list
    else:
        return cell

In [193]:
df["Dehumanization"] = df.pair2.apply(lambda x: replace_with(x, text2)[1])

In [194]:
df["Emotion"] = df.pair3.apply(lambda x: replace_with(x, text3)[1])

In [195]:
df["Mention"] = df['pair1'].apply(lambda x: x[1])

In [197]:
df = df.sort_values(by='External ID')
df.reset_index(inplace=True, drop=True)

# First attempt to calculate with the assumption that annotators are same

In [198]:
df['Rater'] = ['Rater1' if i % 2 == 0 else 'Rater2' for i in range(len(df))]

In [ ]:
from sklearn.metrics import cohen_kappa_score

# define a function to calculate Cohen's kappa for a given column
def calculate_kappa(column_name):
    rater1_labels = df[column_name].loc[df['Rater'] == 'Rater1'].tolist()
    rater2_labels = df[column_name].loc[df['Rater'] == 'Rater2'].tolist()
    kappa = cohen_kappa_score(rater1_labels, rater2_labels)
    return kappa

In [202]:
# calculate Cohen's kappa for the 'Dehumanization' column
dehumanization_kappa = calculate_kappa('Dehumanization')
print("Cohen's kappa for Dehumanization: {:.2f}".format(dehumanization_kappa))

# repeat the above for the 'Emotion' and 'Mention' columns
emotion_kappa = calculate_kappa('Emotion')
print("Cohen's kappa for Emotion: {:.2f}".format(emotion_kappa))

mention_kappa = calculate_kappa('Mention')
print("Cohen's kappa for Mention: {:.2f}".format(mention_kappa))

Cohen's kappa for Dehumanization: 0.50
Cohen's kappa for Emotion: 0.49
Cohen's kappa for Mention: 0.64


# Correct calculation under the real conditions of different annotators

In [248]:
def calculate_cohens_kappa(df, col_name):
    # creating the data for question
    df_column = df[['External ID', 'Created By', col_name]]

    # identifying the overlapping samples and raters who labeled them
    overlapping_samples = df_column.groupby('External ID').filter(lambda x: len(x) > 1)
    unique_sample_ids = overlapping_samples['External ID'].unique()
    rater_pairs = []
    for sample_id in unique_sample_ids:
        raters = overlapping_samples[overlapping_samples['External ID'] == sample_id]['Created By'].tolist()
        rater_pairs.append(raters)

    # averaging the kappa scores for pairs
    kappa_scores = []
    for sample_id, rater_pair in zip(unique_sample_ids, rater_pairs):
        sample_data = overlapping_samples[overlapping_samples['External ID'] == sample_id]
        rater1_labels = sample_data[sample_data['Created By'] == rater_pair[0]][col_name].tolist()[0]
        rater2_labels = sample_data[sample_data['Created By'] == rater_pair[1]][col_name].tolist()[0]
        kappa = cohen_kappa(rater1_labels, rater2_labels)
        kappa_scores.append(kappa)
    mean_kappa_score = sum(kappa_scores) / len(kappa_scores)

    return mean_kappa_score

In [249]:
calculate_cohens_kappa(df, 'Dehumanization')

0.8456889204545459

In [250]:
calculate_cohens_kappa(df, 'Mention')

0.969028693181818

In [251]:
calculate_cohens_kappa(df, 'Emotion')

0.8472082386363637